In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE61281"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE61281"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE61281.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE61281.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE61281.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Human Whole Blood: Psoriatic Arthritis [PsA] vs. Cutaneous Psoriasis Without Arthritis [PsC] vs. Controls"
!Series_summary	"Transcriptional profiling of human whole blood comparing PsA, PsC, and unaffected controls"
!Series_overall_design	"Three condition experiment: PsA, PsC, unaffected controls. Biological replicates: 20 PsA, 20 PsC, 12 controls"
Sample Characteristics Dictionary:
{0: ['tissue: whole blood'], 1: ['condition: Psoriatic arthritis', 'condition: Cutaneous psoriasis without arthritis', 'condition: Unaffected control'], 2: ['gender: Female', 'gender: Male'], 3: ['batch: 4', 'batch: 3', 'batch: 2', 'batch: 1'], 4: ['psoriasis duration: 48.0', 'psoriasis duration: 37.0', 'psoriasis duration: 22.0', 'psoriasis duration: 13.0', 'psoriasis duration: 33.0', 'psoriasis duration: 18.0', 'psoriasis duration: 38.0', 'psoriasis duration: 24.0', 'psoriasis duration: 19.0', 'psoriasis duration: 28.0', 'psoriasis duration: 20.0', 'psoriasis duratio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Dict, Any, Callable
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this seems to be a transcriptional profiling study
# which likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability

# For trait (Psoriatic Arthritis)
# Key 1 contains "condition" which includes our trait of interest
trait_row = 1  

# For gender
# Key 2 contains gender information
gender_row = 2

# For age
# There is no direct age information, only "age of psoriasis onset" and "psoriasis duration"
# We could calculate age, but it's not directly available
age_row = None  

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Optional[int]:
    """Convert trait values to binary format."""
    if value is None:
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check for Psoriatic arthritis vs other conditions
    if "psoriatic arthritis" in value.lower():
        return 1  # Has Psoriatic Arthritis
    elif "cutaneous psoriasis without arthritis" in value.lower() or "unaffected control" in value.lower():
        return 0  # Does not have Psoriatic Arthritis
    else:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary format (0=female, 1=male)."""
    if value is None:
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "female" in value.lower():
        return 0
    elif "male" in value.lower():
        return 1
    else:
        return None

# Age conversion function defined but won't be used as age_row is None
def convert_age(value: str) -> Optional[float]:
    """Convert age values to float."""
    if value is None:
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Using clinical_data that should be available from previous steps
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender,
        age_row=age_row,
        convert_age=convert_age
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1501512': [1.0, 0.0], 'GSM1501513': [1.0, 0.0], 'GSM1501514': [1.0, 1.0], 'GSM1501515': [1.0, 1.0], 'GSM1501516': [1.0, 1.0], 'GSM1501517': [1.0, 1.0], 'GSM1501518': [1.0, 1.0], 'GSM1501519': [1.0, 1.0], 'GSM1501520': [1.0, 1.0], 'GSM1501521': [1.0, 1.0], 'GSM1501522': [1.0, 0.0], 'GSM1501523': [1.0, 0.0], 'GSM1501524': [1.0, 0.0], 'GSM1501525': [1.0, 0.0], 'GSM1501526': [1.0, 0.0], 'GSM1501527': [1.0, 0.0], 'GSM1501528': [1.0, 0.0], 'GSM1501529': [1.0, 1.0], 'GSM1501530': [1.0, 0.0], 'GSM1501531': [1.0, 0.0], 'GSM1501532': [0.0, 1.0], 'GSM1501533': [0.0, 1.0], 'GSM1501534': [0.0, 1.0], 'GSM1501535': [0.0, 1.0], 'GSM1501536': [0.0, 0.0], 'GSM1501537': [0.0, 1.0], 'GSM1501538': [0.0, 1.0], 'GSM1501539': [0.0, 1.0], 'GSM1501540': [0.0, 1.0], 'GSM1501541': [0.0, 1.0], 'GSM1501542': [0.0, 0.0], 'GSM1501543': [0.0, 1.0], 'GSM1501544': [0.0, 0.0], 'GSM1501545': [0.0, 0.0], 'GSM1501546': [0.0, 0.0], 'GSM1501547': [0.0, 0.0], 'GSM1501548': [0.0, 0.0]

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(+)eQC-39', '(+)eQC-40',
       '(+)eQC-41', '(+)eQC-42', '(-)3xSLv1', 'A_23_P100001', 'A_23_P100011',
       'A_23_P100022', 'A_23_P100056', 'A_23_P100074'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Observing the gene identifiers in the gene expression data

# The identifiers seen in the data (like A_23_P100001) appear to be Agilent microarray probe IDs
# rather than human gene symbols. These are proprietary identifiers used on Agilent microarray platforms
# and need to be mapped to standard gene symbols for proper analysis.

# These probe IDs (starting with A_23_P) are a clear indication that we're looking at Agilent array data
# and will require mapping to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain the gene identifiers and gene symbols
# From the preview, it's clear we need 'ID' and 'GENE_SYMBOL'
probe_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

# 2. Get a gene mapping dataframe by extracting the appropriate columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_symbol_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the converted gene expression data
print(f"Number of genes after mapping: {len(gene_data)}")
print("First 10 gene symbols:")
print(gene_data.index[:10])


Number of genes after mapping: 18488
First 10 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
# Note: We already normalized the gene data in the previous step
normalized_gene_data = gene_data.copy()
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")

Gene data saved to ../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE61281.csv
Linked data shape before handling missing values: (52, 18490)


Linked data shape after handling missing values: (52, 18490)
For the feature 'Psoriatic_Arthritis', the least common label is '1.0' with 20 occurrences. This represents 38.46% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 24 occurrences. This represents 46.15% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data is usable. Saving to ../../output/preprocess/Psoriatic_Arthritis/GSE61281.csv
